https://jira.x5.ru/browse/CVMUPG-797

In [1]:
name = 'CVMUPG-797'

In [2]:
import os
import re
from pyspark.sql import SparkSession
from typing import List, Dict, Callable
import socket

spark = None

EXECUTOR_ENV = 'hdfs:///share/products/cvm5/lib/python/anaconda_2.4.4_ds.tar.gz'  # 2.4.4 
SPARK_ARCHIVE = 'hdfs:///share/lib/spark/sparkjars-2.4.4.zip'                     # 2.4.4
#EXECUTOR_ENV = 'hdfs:///share/lib/python/env/anaconda-2019.07.tar.gz'
#SPARK_ARCHIVE = 'hdfs:///share/lib/spark/sparkjars-2.3.1.zip'

os.environ["ARROW_LIBHDFS_DIR"] = "/usr/hdp/2.6.5.0-292/usr/lib"
os.environ['HADOOP_HOME'] = '/usr/hdp/current/hadoop-client/'
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64/'
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf/'
os.environ['SPARK_HOME'] = '/opt/conda/lib/python3.7/site-packages/pyspark'
os.environ['PYSPARK_PYTHON'] = 'anaconda_2.4.4_ds.tar.gz/bin/python3'             # 2.4.4
#os.environ['PYSPARK_PYTHON'] = 'anaconda-2019.07.tar.gz/bin/python3'






def restart_spark(task_name: str, num_executors: int, executor_memory='4G', executor_cores=2,
                  driver_memory='2G', queue='bdse', additional_params: Dict[str, str] = None):
    global spark

    if spark:
        sc = spark.sparkContext
        if sc and sc._jsc:
            if not sc._jsc.sc().isStopped():
                print('Using cached spark')
                return sc, spark

    need_ports_for_app = 3
    user_tcp_ports = _get_user_tcp_ports()
    free_ports = _get_free_ports(user_tcp_ports)
    assert len(free_ports) >= need_ports_for_app, \
        f"Not enough free ports ({len(free_ports)}), need {need_ports_for_app}, stop other apps"
    app_ports = free_ports[:need_ports_for_app]

    host_ip = os.getenv('HOST_IP')
    
    spark_session = (
        SparkSession
        .builder
        .appName(task_name)
        .master('yarn')
        .config('spark.driver.memory', driver_memory)
        .config('spark.driver.maxResultSize', driver_memory)
        .config('spark.executor.cores', executor_cores)
        .config('spark.executor.memory', executor_memory)
        .config('spark.executor.memoryOverhead', '1G')
        .config('spark.dynamicAllocation.enabled', 'true')
        .config('spark.dynamicAllocation.maxExecutors', num_executors)
        .config('spark.sql.broadcastTimeout', '36000')
        .config('spark.dynamicAllocation.cachedExecutorIdleTimeout', '1200s')
        .config('spark.ui.port', app_ports[0])
        .config('spark.blockManager.port', app_ports[1])
        .config('spark.driver.port', app_ports[2])
        .config('spark.driver.host', host_ip)
        .config('spark.driver.bindAddress', '0.0.0.0')
        .config('spark.driver.extraLibraryPath', '/usr/hdp/2.6.5.0-292/hadoop/lib/native')
        .config('spark.driver.extraJavaOptions', '-Dhdp.version=current')
        .config('spark.debug.maxToStringFields', '50')
        .config('spark.yarn.queue', queue)
        .config('spark.yarn.dist.archives', EXECUTOR_ENV)
        .config('spark.yarn.archive', SPARK_ARCHIVE)
        .config('spark.yarn.am.extraJavaOptions', '-Dhdp.version=current')
        .config('spark.rpc.message.maxSize', '1024')
        .config('spark.sql.warehouse.dir', '/apps/hive/warehouse')
        .config('spark.sql.execution.pandas.respectSessionTimeZone', 'false')
        .config('spark.sql.orc.filterPushdown', 'true')
        .config('spark.sql.hive.convertMetastoreOrc', 'true')
        .config('spark.shuffle.service.enabled', 'true')
        .config('spark.hadoop.yarn.timeline-service.enabled', 'false')
        .config('spark.hadoop.yarn.client.failover-proxy-provider',
                'org.apache.hadoop.yarn.client.ConfiguredRMFailoverProxyProvider')
        .config('spark.serializer', 'org.apache.spark.serializer.KryoSerializer')
        .config('spark.kryoserializer.buffer.max', '128m')
        .config('spark.executor.extraLibraryPath', '/usr/hdp/2.6.5.0-292/hadoop/lib/native')
    )

    if additional_params:
        for key, value in additional_params.items():
            spark_session = spark_session.config(key, value)

    spark = (
        spark_session
        .enableHiveSupport()
        .getOrCreate()
    )
    sc = spark.sparkContext

    return sc, spark


def _get_user_tcp_ports() -> List[str]:
    regexp = re.compile(r'-2e')
    envuser= os.getenv('HOSTNAME')
    if regexp.search(envuser):       
      _, user_name, user_surname = envuser.upper().split('-')
      user_full_name = '_'.join([user_name, user_surname])
    else:
      _, user_name  = envuser.upper().split('-') 
      user_full_name = user_name
    user_tcp_ports = [v for k, v in os.environ.items() if user_full_name in k and k.endswith('TCP_PORT')]
    return user_tcp_ports


def _get_free_ports(ports: List[str]):
    free_ports = []
    for port in ports:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            if s.connect_ex(('0.0.0.0', int(port))) != 0:
                free_ports.append(port)
    return free_ports


In [3]:
sc, spark = restart_spark(
                    name, 
                    21, 
                    executor_memory='5G', 
                    executor_cores=3, 
                    driver_memory='7G', 
                    additional_params={"spark.sql.shuffle.partitions": "300"})

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
sc.setLogLevel('ERROR')

In [5]:
import pandas as pd
import sys
from pyspark.sql import functions as F
import datetime
from datetime import timedelta

sys.path.append('/home/jovyan/glow-byte-filters-pyspark')
from logic_filters import * 

In [6]:
LOYALTY_CARDS = "hive_ssa_tc5.loyalty_card"
LOYALTY_CARDHOLDERS = "hive_ssa_tc5.loyalty_cardholder"
ACCOUNTS = "hive_ssa_tc5.account"
CVM5_GUESTS = "hive_cvm_acrm.cvm5_guests"

DIM_STORE = "hive_ssa_main.dim_store"
CHECKS_HEADERS = "hive_ssa_main.fct_rtl_txn"
CHECKS_ITEMS = "hive_ssa_main.fct_rtl_txn_item"
PRODUCTS = "hive_ssa_tc5.cvm_product"

# seg1

### Выбираем гостей нужного юзкейса

In [7]:
usecase = ['cross', 'upgrade', 'ump']
dt = datetime.date.today()

In [8]:
customers_usecase = (spark
                     .table(CVM5_GUESTS)
                     .filter(F.col('calculation_dt') >= dt)
                     .filter(F.col('usecase').isin(usecase))
                     .select('account_no', 'customer_rk')
                            )

### По этой активной базе отсекаем ГКГ, проверяем доступность в канале смс вайбер на 25 ноября.

In [9]:
check_date = '2021-11-25'

In [10]:
guests = remove_global_control_group(spark, customers_usecase, check_date)

 Фильтр Глобальная контрольная группа - валиден.


[Stage 6:======================================================>(299 + 1) / 300]

	 Отсёк 1 354 431 записей.


In [11]:
seg_sms1 = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=guests, 
                                         usecase_name=usecase[0], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:07


In [12]:
seg_sms2 = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=guests, 
                                         usecase_name=usecase[1], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:05


In [13]:
seg_sms3 = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=guests, 
                                         usecase_name=usecase[2], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:08


In [14]:
seg_sms = seg_sms1.union(seg_sms2).union(seg_sms3).distinct()

In [15]:
seg_sms.write.parquet('temp11', mode='overwrite')
seg_sms = spark.read.parquet('temp11')

In [16]:
seg_sms.count()

2183084

### Собираем чеки этих гостей

In [17]:
start_date = datetime.date(2021, 8, 1)
end_date = datetime.date(2021, 11, 1)

In [18]:
checks_headers = (spark.table(CHECKS_HEADERS)
                      .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                      .filter((F.col('loyalty_card_no') != '') & (F.col('loyalty_card_no').isNotNull()))
                      .filter(F.col('financial_unit_format_dk') == 'D')
                      .filter(F.col('rtl_txn_cancel_flg') == 0)
                      .select('rtl_txn_id', 'loyalty_card_no')
                     )


In [19]:
loyalty_cards = (spark
                    .table(LOYALTY_CARDS)
                    .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                    .withColumnRenamed("loyalty_account_id", "account_no")
                    .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                    .select('account_no', 'loyalty_card_no')
                )
loyalty_cardholders = (spark
                        .table(LOYALTY_CARDHOLDERS)
                        .filter(((F.col('loyalty_cardholder_age_yrs') >= 25) & 
                                (F.col('loyalty_cardholder_age_yrs') <= 65)) | 
                                (~F.col('loyalty_cardholder_age_yrs').isNotNull()))
                        .filter(F.col('gender_dk') == 'F')
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info = clients_info.join(seg_sms, on='customer_rk', how='inner')

In [20]:
loyalty_cardholders2 = (spark
                        .table(LOYALTY_CARDHOLDERS)
                        .filter((F.col('loyalty_cardholder_age_yrs') >= 25) & 
                                (F.col('loyalty_cardholder_age_yrs') <= 65))
                        .filter(F.col('gender_dk') == 'M')
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info2 = loyalty_cards.join(loyalty_cardholders2, on='account_no', how='inner')
clients_info2 = clients_info2.join(seg_sms, on='customer_rk', how='inner')

In [21]:
checks_headers_tc5 = checks_headers.join(clients_info, on='loyalty_card_no') #оставили чеки только нужных гостей

In [22]:
checks_headers_tc5_2 = checks_headers.join(clients_info2, on='loyalty_card_no') #оставили чеки только нужных гостей

In [23]:
checks_items = (spark.table(CHECKS_ITEMS) 
                    .withColumnRenamed('plu_id', 'plu_code')
                    .withColumnRenamed('turnover_no_vat_amt', 'zsalnovat')
                    .withColumnRenamed('turnover_vat_rub_amt', 'zsale_vat')
                    .withColumnRenamed('prime_cost_no_vat_amt', 'zcst_n')
                    .withColumnRenamed('turnover_base_uom_amt', 'base_qty')
                    .withColumnRenamed('discount_amt', 'zdiscount')
                    .withColumnRenamed('fact_regular_promo_flg', 'zpromofl')
                    .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                    .filter((F.col('zsalnovat') >= 0) & (F.col('base_qty') >= 0) & (F.col('zcst_n') > 0)) #keep only correct data
                        .select('plu_code' #items id
                                , 'rtl_txn_id' #cheques id
                               )
                   )


#### Отбираем тех, кто покупал продукты в категории масло

##### Отбираем PLU определенной категории (масло растительное) 1 л

In [24]:
plu_codes_cat = (spark
                 .table(PRODUCTS)
#                  .filter(F.col('plu_hierarchy_lvl_4_dk') == plu_hierarchy_lvl_4_dk)
#                  .filter(F.col('plu_brand_code').isin(plu_brand_code))
                 .filter(F.col('syntethic_category_id').isin([74]))
#                  .filter(~F.col('plu_id').isin(plu_not_in))
                 .select('plu_id')
                 .distinct()
                 .toPandas()['plu_id']
                 .tolist()
                )

In [25]:
checks_items = checks_items.filter(F.col('plu_code').isin(plu_codes_cat)) # только чеки с нужными plu

In [26]:
checks_tc5 = checks_items.join(checks_headers_tc5, on='rtl_txn_id', how='inner')

In [27]:
checks_tc5_2 = checks_items.join(checks_headers_tc5_2, on='rtl_txn_id', how='inner')

In [44]:
seg_women = checks_tc5.withColumn('gender', F.lit('F')).select('customer_rk', 'gender').distinct()

In [45]:
seg_men = (checks_tc5_2
           .withColumn('gender', F.lit('M'))
           .select('customer_rk', 'gender')
           .distinct()
      )

In [46]:
seg_union = seg_women.union(seg_men)

In [47]:
seg_union.write.parquet('temp12', mode='overwrite')
seg_union = spark.read.parquet('temp12')

In [48]:
seg_union.count()

498746

#### Сегмент потребителей масла растительного готов

#### Отбираем тех, кто покупал продукты брендов

In [49]:
loyalty_cards = (spark
                    .table(LOYALTY_CARDS)
                    .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                    .withColumnRenamed("loyalty_account_id", "account_no")
                    .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                    .select('account_no', 'loyalty_card_no')
                )
loyalty_cardholders = (spark
                        .table(LOYALTY_CARDHOLDERS)
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info = clients_info.join(seg_union, on='customer_rk', how='inner')

In [50]:
checks_headers_tc5 = checks_headers.join(clients_info, on='loyalty_card_no') #оставили чеки только нужных гостей

In [51]:
checks_items = (spark.table(CHECKS_ITEMS) 
                    .withColumnRenamed('plu_id', 'plu_code')
                    .withColumnRenamed('turnover_no_vat_amt', 'zsalnovat')
                    .withColumnRenamed('turnover_vat_rub_amt', 'zsale_vat')
                    .withColumnRenamed('prime_cost_no_vat_amt', 'zcst_n')
                    .withColumnRenamed('turnover_base_uom_amt', 'base_qty')
                    .withColumnRenamed('discount_amt', 'zdiscount')
                    .withColumnRenamed('fact_regular_promo_flg', 'zpromofl')
                    .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                    .filter((F.col('zsalnovat') >= 0) & (F.col('base_qty') >= 0) & (F.col('zcst_n') > 0)) #keep only correct data
                        .select('plu_code' #items id
                                , 'rtl_txn_id' #cheques id
                               )
                   )

In [52]:
plu_codes_cat2 = (spark
                 .table(PRODUCTS)
#                  .filter(F.col('plu_hierarchy_lvl_4_dk') == plu_hierarchy_lvl_4_dk)
                 .filter(F.col('plu_brand_code').isin(['9347', '8027', '9346', '9336', 'B980']))
#                  .filter(~F.col('plu_id').isin(plu_not_in))
                 .select('plu_id')
                 .distinct()
                 .toPandas()['plu_id']
                 .tolist()
                )

In [53]:
checks_items = checks_items.filter(F.col('plu_code').isin(plu_codes_cat2)) # только чеки с нужными plu

In [54]:
checks_tc5 = checks_items.join(checks_headers_tc5, 'rtl_txn_id', how='inner')

In [60]:
seg_union_brand = (checks_tc5.select('customer_rk', 'gender')
                  .distinct()
      )

In [61]:
seg_union_brand.write.parquet('temp13', mode='overwrite')
seg_union_brand = spark.read.parquet('temp13')

In [62]:
seg_union_brand.count()

375545

In [63]:
seg_pd = seg_union.toPandas()

In [64]:
seg2_pd = seg_union_brand.toPandas()

21/11/24 15:12:36 ERROR client.TransportClient: Failed to send RPC RPC 6425125769648841476 to /192.168.234.57:53798: java.nio.channels.ClosedChannelException
java.nio.channels.ClosedChannelException
	at io.netty.channel.AbstractChannel$AbstractUnsafe.write(...)(Unknown Source)
21/11/24 15:12:36 ERROR client.TransportClient: Failed to send RPC RPC 7768892084552438452 to /192.168.234.57:53798: java.nio.channels.ClosedChannelException
java.nio.channels.ClosedChannelException
	at io.netty.channel.AbstractChannel$AbstractUnsafe.write(...)(Unknown Source)
21/11/24 15:12:36 ERROR cluster.YarnScheduler: Lost executor 117 on mn-hd0172.x5.ru: Slave lost
21/11/24 15:12:36 ERROR cluster.YarnScheduler: Lost executor 116 on mn-hd0127.x5.ru: Slave lost
21/11/24 15:12:36 ERROR client.TransportClient: Failed to send RPC RPC 7654263213903650146 to /192.168.234.57:53798: java.nio.channels.ClosedChannelException
java.nio.channels.ClosedChannelException
	at io.netty.channel.AbstractChannel$AbstractUnsafe.w

In [66]:
seg_pd

,customer_rk,gender
0,23820760,F
1,125058996,F
2,14675917,F
3,45718683,F
4,22682921,F
...,...,...
498741,34532354,M
498742,97862792,M
498743,37633574,M
498744,13277450,M


In [67]:
seg1_women = seg_pd[seg_pd['gender'] == 'F']

In [68]:
seg1_men = seg_pd[seg_pd['gender'] == 'M']

In [69]:
seg2_women = seg2_pd[seg2_pd['gender'] == 'F']
seg2_men = seg2_pd[seg2_pd['gender'] == 'M']

In [70]:
print('seg1_f: {}, seg1_m: {}, seg2_f: {}, seg2_m: {}'.format(len(seg1_women), 
                                                              len(seg1_men), len(seg2_women), len(seg2_men)))

seg1_f: 308083, seg1_m: 190663, seg2_f: 231646, seg2_m: 143899


In [71]:
def on_usecases(df, usecases='standart'):
    if usecases == 'standart':
        cross = (df.loc[(df['customer_rk'].str.endswith('1')) | (df['customer_rk'].str.endswith('2'))
                  | (df['customer_rk'].str.endswith('3')) | (df['customer_rk'].str.endswith('4'))].reset_index(drop=True))
        upgrade = (df.loc[(df['customer_rk'].str.endswith('5')) | (df['customer_rk'].str.endswith('6'))
                  | (df['customer_rk'].str.endswith('7')) | (df['customer_rk'].str.endswith('8'))].reset_index(drop=True))
        ump = df.loc[(df['customer_rk'].str.endswith('0')) | (df['customer_rk'].str.endswith('9'))].reset_index(drop=True)

    return cross, upgrade, ump

In [72]:
seg_no = 'seg1_f'

In [73]:
seg1_women.to_csv(name + '_' + seg_no + '_All_uc' + str(dt) + '.csv', index=False)

In [75]:
seg1_women = seg1_women.astype('str')

In [76]:
cross, upgrade, ump = on_usecases(seg1_women)

In [77]:
cross.to_csv(name + '_' + seg_no + '_' + usecase[0] + '_' + str(dt) + '.csv', index=False)

In [78]:
upgrade.to_csv(name + '_' + seg_no + '_' + usecase[1] + '_' + str(dt) + '.csv', index=False)

In [79]:
ump.to_csv(name + '_' + seg_no + '_' + usecase[2] + '_' + str(dt) + '.csv', index=False)

In [80]:
print('seg_no: {}, cross: {}, upgrade: {}, ump: {}'.format(seg_no, len(cross), len(upgrade), len(ump)))

seg_no: seg1_f, cross: 106441, upgrade: 117130, ump: 84512


In [81]:
seg_no = 'seg1_m'

In [82]:
seg1_men.to_csv(name + '_' + seg_no + '_All_uc' + str(dt) + '.csv', index=False)

In [83]:
seg1_men = seg1_men.astype('str')

In [84]:
cross, upgrade, ump = on_usecases(seg1_men)

In [85]:
cross.to_csv(name + '_' + seg_no + '_' + usecase[0] + '_' + str(dt) + '.csv', index=False)

In [86]:
upgrade.to_csv(name + '_' + seg_no + '_' + usecase[1] + '_' + str(dt) + '.csv', index=False)

In [87]:
ump.to_csv(name + '_' + seg_no + '_' + usecase[2] + '_' + str(dt) + '.csv', index=False)

In [88]:
print('seg_no: {}, cross: {}, upgrade: {}, ump: {}'.format(seg_no, len(cross), len(upgrade), len(ump)))

seg_no: seg1_m, cross: 64182, upgrade: 69633, ump: 56848


In [89]:
seg_no = 'seg2_f'

In [90]:
seg2_women.to_csv(name + '_' + seg_no + '_All_uc' + str(dt) + '.csv', index=False)

In [91]:
seg2_women = seg2_women.astype('str')

In [92]:
cross, upgrade, ump = on_usecases(seg2_women)

In [93]:
cross.to_csv(name + '_' + seg_no + '_' + usecase[0] + '_' + str(dt) + '.csv', index=False)

In [94]:
upgrade.to_csv(name + '_' + seg_no + '_' + usecase[1] + '_' + str(dt) + '.csv', index=False)

In [95]:
ump.to_csv(name + '_' + seg_no + '_' + usecase[2] + '_' + str(dt) + '.csv', index=False)

In [96]:
print('seg_no: {}, cross: {}, upgrade: {}, ump: {}'.format(seg_no, len(cross), len(upgrade), len(ump)))

seg_no: seg2_f, cross: 79548, upgrade: 87090, ump: 65008


In [97]:
seg_no = 'seg2_m'

In [98]:
seg2_men.to_csv(name + '_' + seg_no + '_All_uc' + str(dt) + '.csv', index=False)

In [99]:
seg2_men = seg2_men.astype('str')

In [100]:
cross, upgrade, ump = on_usecases(seg2_men)

In [101]:
cross.to_csv(name + '_' + seg_no + '_' + usecase[0] + '_' + str(dt) + '.csv', index=False)

In [102]:
upgrade.to_csv(name + '_' + seg_no + '_' + usecase[1] + '_' + str(dt) + '.csv', index=False)

In [103]:
ump.to_csv(name + '_' + seg_no + '_' + usecase[2] + '_' + str(dt) + '.csv', index=False)

In [104]:
print('seg_no: {}, cross: {}, upgrade: {}, ump: {}'.format(seg_no, len(cross), len(upgrade), len(ump)))

seg_no: seg2_m, cross: 47988, upgrade: 52002, ump: 43909
